<a href="https://colab.research.google.com/github/EventHopper/ReviewsScraping/blob/master/FBL_App_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FB Local Competitive Review**


---


### **Application**
**Facebook Local**

Details:

Size: 200 MB+,
Installs: 5M+,
Requires Android 4.1+

---



### **Review Rationale**

*We want to review feedback for this app on the Google Play Store and AppStore. Note that both negative and positive are good, however the negative reviews can reveal critical features that are missing or significant bugs (if they appear frequently).*
<br>

### **Packages Used**

**Google Play Store Related Packages:**

 - [google-play-scraper](https://github.com/JoMingyu/google-play-scraper)

**AppStore Related Packages:**

- TBA (to be added)

# Project Init

In [0]:
!pip install -qq google-play-scraper

     |████████████████████████████████| 51kB 1.6MB/s 


In [0]:
!pip install -qq -U watermark

In [0]:
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

CPython 3.6.9
IPython 5.5.0

pandas 1.0.4
matplotlib 3.2.1
seaborn 0.10.1
google_play_scraper 0.0.2.6


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import json
import time
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

# Data Wrangling

## Google Play Store Scraping

Step Summary:

- Scrape Google Play app information
- Scrape user reviews for Google Play apps
- Save the dataset to CSV files

Package name of app: com.facebook.Socal
Obtained using ADB shell command


```
$ adb shell "pm list packages -f facebook"
```

In [0]:
app_packages = [
  'com.facebook.Socal'
]

Utilized package name in for loop (note multiple package names could have been added to app_packages for same result)

In [0]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


We got the info for the app. Let's write a helper function that prints JSON objects a bit better:

In [0]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

Here is a sample app information from the list:

In [0]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "4.1",
  "androidVersionText": "4.1 and up",
  "appId": "com.facebook.Socal",
  "containsAds": null,
  "contentRating": "Teen",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Welcome to Facebook Local!\u00a0We\u2019ve updated our events app to help you discover places and things to do recommended by friends you know and trust.\r\n\r\nKeep up with what\u2019s happening locally\u2014wherever you are\u2014whether you\u2019re looking for something to do with friends this weekend or want to explore a new neighborhood.\r\n\r\nFeatures:\r\n\r\n* See the most recent activity, events and places your friends are interacting with and updates from event hosts and Pages you follow.\r\n* Find events and activities happening near you on an interactive map and filter by time, category, location and more.\r\n* Get recommendations based on what's popular with your friends, things you've been to in the past, Pages you like and our c

This contains lots of information including the number of ratings, number of reviews and number of ratings for each score (1 to 5). Let's ignore all of that and have a look at their beautiful icons:

In [0]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

fig, axs = plt.subplots(2, len(app_infos) // 2, figsize=(14, 5))

for i, ax in enumerate(axs.flat):
  ai = app_infos[i]
  img = plt.imread(ai['icon'])
  ax.imshow(img)
  ax.set_title(format_title(ai['title']))
  ax.axis('off')

<Figure size 1008x360 with 0 Axes>

In [0]:
timestr = time.strftime("%Y%m%d-%H%M%S")
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('app_details'+timestr+'.csv', index=None, header=True)

### Scraping App Reviews

In an ideal world, we would get all the reviews. But there are lots of them and we're scraping the data. That wouldn't be very polite. What should we do?

We want:

- Balanced dataset - roughly the same number of reviews for each score (1-5)
- A representative sample of the reviews for each app

We can satisfy the first requirement by using the scraping package option to filter the review score. For the second, we'll sort the reviews by their helpfulness, which are the reviews that Google Play thinks are most important. Just in case, we'll get a subset from the newest, too:

In [0]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:03<00:00,  3.52s/it]


In [0]:
print_json(app_reviews[0])

{
  "appId": "com.facebook.Socal",
  "at": "2020-04-22 11:54:28",
  "content": "Uses 496MB internal memory and won't run from SD. Poor quality assurance - some businesses appear randomly dropped in entirely the wrong place (e.g. Costa, Manchester Arndale Centre is shown on a housing estate near Bolton). Uninstalled.",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": "19.0.0.52.2104",
  "reviewId": "gp:AOqpTOGGWG0mw15Cg7v0CXrB2FRJ5vToBK6ndfUuDGDfyPdmc5r6ya5g6pCI7SimADOzWuExJ7aR01WemP-g_zQ",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 256,
  "userImage": "https://lh3.googleusercontent.com/-62mHJxvVL2o/AAAAAAAAAAI/AAAAAAAAAAA/AMZuucnrlZzdPDn2zp6QN_DRqhkPnsFupg/photo.jpg",
  "userName": "Tom Higgs"
}



In [0]:
len(app_reviews)

1120

In [0]:
timestr = time.strftime("%Y%m%d-%H%M%S")
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('FacebookLocalReviews-'+timestr+'.csv', index=None, header=True)

20200606-043144


## AppStore Scraping

In [0]:
# TBA

# EDA

Next, we're going to use the reviews for sentiment analysis with BERT. But first, we'll have to do some text preprocessing!

In [0]:
# TBA

# Summary

TBA

# References


- [Brilliant tutorial for Google Play Store Scraping](https://www.curiousily.com/posts/create-dataset-for-sentiment-analysis-by-scraping-google-play-app-reviews-using-python/)